**Oma andmetöötluse kodutöös uurime andmeid tulnukate kohta ehk täpsemalt:**
* loome kokkuvõtte tabeli gruppide kaupa, et näidata, mis kujul algselt andmeid kogusime ja kuidas neid analüüsisime
* loome histogrammi, mis näitab mitu sekundit on keskmiselt ufonaudid kellegi silmnähtavuses viibinud
* loome scatterploti, mis näitab millistel longituudidel ja latituudidel on ufosid kõige rohkem kohatud
* loome pie charti selle kohta, mis riikides on kõige rohkem tulnukaid esinenud*

Esimese asjana impordime andmete analüüsimiseks vajaminevad teegid.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pandas import read_csv

%matplotlib inline
pd.set_option('display.max_rows', 20)

Pandas oskab lugeda paljusid erinevaid andmeformaate (näiteks Excel ja CSV) nii kohalikult kettalt kui veebist. Antud juhul loeme sisse ühe CSV faili kohalikult kettalt. Antud juhul on meie algandmed CSV formaadis ja seetõttu kasutame moodulis pandas olevat funktsiooni read_csv, mis võtab argumendiks failinime või URL-i, loeb andmed mällu ning tagastab objekti, mille kaudu saab andmeid uurida. Kaggle korraldab vaikimisi nii, et märkmikuga seotud andmed on kättesaadavad jooksva kausta ülemkaustas (../) olevas alamkaustas nimega input.

In [ ]:
df = pd.read_csv("../input/ufo_sighting_data.csv")

Objekt, mida read_csv tagastas ja mille me salvestasime muutujasse df, on tüübiga pandas.DataFrame, mis ongi antud juhendi keskmes. (df asemel oleksime võinud kasutada suvalist muutujanime, aga df on mõnusalt lühike ja annab ka lugejale kohe märku, et tegemist on DataFrame'iga.) Laseme kõigepealt Notebook-il seda objekti meile näidata. Notebook-i koodiplokkidel on selline omadus, et kui ploki viimaseks elemendiks on mingi avaldis, siis ploki käivitamisel näidatakse selle avaldise väärtust.

In [ ]:
df

Super! Asi töötab! Uurime oma andmete kogumit lähemalt järgmise koodijupiga.

In [ ]:
df.info()

Huvitav mitu rida meie andmete kogumis on? Ka seda saab järele uurida!

In [ ]:
len(df)

Hakkame vaikselt leiutama pie charti riikide kohta, et kus kõige rohkem tulnukad hiilimas on käinud. Kuvame esiteks huvi pärast tabeli kõikide riikidega ja seejärel uurime .value_counts() käsuga, mis riikides ufode esinemisi on kõige rohkem olnud. 

*NB! Paneme tähele, et osade tulnukate märkamise andmete kohta on kogumikus märgitud riigi alla "NaN" ehk täpsustamata ja neid me oma pie chartis ei kuva.*

In [ ]:
df["country"]

In [ ]:
df["country"].value_counts()

Kasutame eelpool leitud andmeid pie charti tegemiseks (mille jaoks vajame matplotlib teeki).

In [ ]:
labelid = ["USA","CANADA","UK","AUSTRAALIA","SAKSAMAA"]

matplotlib.pyplot.pie(df["country"].value_counts(),labels=labelid,radius=3.5)

Näeme oma koogi pealt, et domineerivalt on ufosid märgatud Ameerika Ühendriikides. Kõige vähem on neid olnud Saksamaal, mille sektorit on raske silmaga eristada isegi nägu vastu ekraani pannes.

Liigume edasi scatterploti juurde ehk uurime millistel longituudidel ja latituudidel on ufosid kõige tihedamini märgatud.

In [ ]:
uus1 = pd.to_numeric(pd.Series(df["latitude"]), errors='coerce')
uus2 = pd.to_numeric(pd.Series(df["longitude"]), errors='coerce')
plt.scatter(uus1, uus2, 5, alpha=0.2)

Mõned lihtsamad näited :

* **California, USA** - 36.778259 : -119.417931
* **San Antonio, Texas, USA** : 29.424122, -98.493629
* **Lincoln, Nebraska, USA** : 40.858449, -96.727122
* **New York, USA** : 40.7127837, -74.005941
* **Berlin, Saksamaa **: 52.520008, 13.404954
* **Vancouver, Kanada** : 49.246292, -123.116226

Järgmiseks üritame luua histogrammi kasutades iga nähtud ufo kohta seda kui kaua ajaliselt neid märgati. Ühikud on sekundites. Teoreetiliselt peaks meil tulema histogramm, kus x-telg näitab ühtlasteks osadeks jaotatud sekundeid ja y-telg näitab sagedust, et kui palju juhtumeid sellesse konkreetsesse sekundite vahemikku on jäänud. See on juhul kui programm töötab ideaalselt.

Reaalsuses, tekib histogramm, kus millegipärast on x-teljel siltideks kõigi erinevate vahemikkude nimetused (üritasime seda eemaldada erinevaid matplotlib ja pandas-e enda käskluseid kasutades, kuid tulutult), seega sealt pole eriti midagi aru saada. Suutsime neid nimetusi vähemalt pöörata 90 kraadi, et natukenegi midagi lugeda saaks ja nii on näha, et kui arv on suurem oma kuju poolest, siis järelikult ongi tegu suurema arvuga. :) Teiseks, arvud pole otseselt meile vajalikus järjestuses. Programm saab aru, et on vaja histogrammi vastava arvu osahulkadega ja joonisel täpselt nii ongi, aga x-teljel on arvud väga imelikult jaotunud - nt 6.1, 65000, 7.1 jne... 7.1 ei ole suurem kui 65000 aga praegu nii histogrammis seisab. Selle vältimiseks proovisime anda sekundite veergu algsest datasetist matplotlibile ette järjestatud kujul, kuid siin tekkis huvitav olukord. Põhimõtteliselt datasetis on osa sekundeid vist vigaselt kirjas, nt ühes kohas on "2" sekundi asemel kirjas "2' " ja on ka mõningaid muid näpuvigu. Arvude järjestamiseks peab enne vastava veeru tõlgendama sõnedest arvudeks, sest muidu programm ei oska sõnesid (vigaseid sisendeid) võrrelda arvudega. Sellise tõlgenduse tegemiseks on olemas käsklus, mille korral toimib selline konvertimine ainult juhul kui imelike sisendite puhul need jäetakse vahele ja nende asemel võetakse sisendiks "NaN". matplotlib kahjuks ei oska "NaN" väärtusega midagi peale hakata ja annab suvalise errori stiilis, et maximum range must be higher than minimum range. Seetõttu jääb hetkel lahendus nii nagu ta on, sest histogrammi sekundite põhjal on niivõinaa väga raske teha, arvestades, et vähim sekundite arv, mil ühel korral ufosid nähti on 0.001 sekundit ja kõige suurem oli umbes 98 miljonit ehk 51 tundi järjest (seems legit :P). Võib-olla lähitulevikus huvi pärast uurime edasi histogrammi kohta, et kuidas seda normaalselt esitada, aga hetkel käib see kah.

In [ ]:
uus = pd.to_numeric(pd.Series(df["length_of_encounter_seconds"]), errors='coerce', downcast='integer')
uus_sorted = uus.sort_values()

plt.figure(figsize=(40,20))
plt.locator_params(axis='x', numticks=10)
plt.xticks(fontsize=16, rotation=90)
plt.hist(df["length_of_encounter_seconds"], bins=100)

Viimaseks koodijupiks oleme valmistanud kokkuvõtva tabeli nendest veergudest, mida kasutasime selles Notebookis analüüsimiseks. Need on küll pmst üks-ühele algsest dataset-ist võetud, kuid näitame siinkohal oskust, et suudame ka ise oma datasette veerghaaval kokku panna. :)

Ühtlasi on siin ka arvutatud mediaan latituud/longituud ja mediaan ufodega kohtumise kestvus.

In [ ]:
miinlat = uus1.mean()
miinlon = uus2.mean()
miinen = uus.mean()
print("Latitude'i mean: " + str(miinlat))
print("Longitude'i mean: " + str(miinlon))
print("Encounter'i mean: " + str(miinen))

#fd = pd.DataFrame({"Latitue mean" : miinlat,
#             "Longitude mean" : miinlon, "Encounter time mean": miinen})
fd = pd.DataFrame({"Latitue" : df["latitude"],
             "Longitude" : df["longitude"], "Encounter time": df["length_of_encounter_seconds"]})
fd


Selleks korraks ongi kõik! Olge mõnusad :)